In [10]:
%%capture
%run 02_account.ipynb
%run 03_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes.

Pow uses computational ressources to calculate a specific hash. This can only be done by brute force. 

In [2]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[:diff] == '0'*diff: break
            nonce += 1
        bh.diff   = diff
        bh.reward = reward
        bh.nonce  = nonce
        bh.miner  = self.pub
        bh.mined  = True
        return bh
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [3]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, rh(), 2, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

hash:		💣 3d2560fcbf37d6c1...0a9
time:		Mon Mar 29 21:03:26 2021
number:		2
prev_hash:	🔹 d36385f32a7f4301...d49
txs_count:	2
txs_root:	🕢 fc58bfdf4d482ccf...237


### Mining

Mine a block on top of the genesis block

In [4]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.bh, 2, 100); print(mb)

hash:		💇 211f3bdff551f265...4d0
time:		Mon Mar 29 21:03:26 2021
number:		3
prev_hash:	💣 3d2560fcbf37d6c1...0a9
txs_count:	11
txs_root:	🔈 a2ff8e93bb84621e...c10
miner:		🕓 0xeD729d8c744999...E9f
diff:		2
reward:		100
nonce:		28
